In [1]:
#import geopandas as gpd
import pandas as pd

## 1. vorgefilterte Daten einlesen
enthalten nur Angaben zur Stadt St.Gallen, Datenquelle = daten.sg.ch

Die Daten zur **Lärmbelastung** enthalten Angaben zum berechneten Lärm an der Fassade. Daher enthält der Datenansatz auch mehr oder weniger vollständige Angaben zur Nutzung. 

Als Abgrenzung: Der Datensatz Lärmemission enthält Angaben zum Lärm am Strassenrand.


In [46]:
df = pd.read_csv('daten/larmbelastung-csv.csv', sep = ';')

In [47]:
df.head(3)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Lr Tag [dB(A)],Immissionswert Lr Nacht [dB(A)],Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht
0,"47.4225523835,9.35814260469","{""coordinates"": [[[9.35820133194473, 47.422598...",St. Gallen,9000,St. Gallen,58.0,45.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW
1,"47.4375957306,9.37816659637","{""coordinates"": [[[9.378254248447286, 47.43760...",St. Gallen,9010,St. Gallen,58.0,43.0,ES III,Wohnen,≤PW,≤PW,≤PW
2,"47.4165304538,9.37865959965","{""coordinates"": [[[9.378730043296102, 47.41652...",St. Gallen,9000,St. Gallen,58.0,43.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW


In [48]:
df.rename({
    'Immissionswert Lr Tag [dB(A)]' : 'Immissionswert Tag',
    'Immissionswert Lr Nacht [dB(A)]' : 'Immissionswert Nacht'
    }, axis = 1, inplace = True)

In [49]:
df.head(3)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht
0,"47.4225523835,9.35814260469","{""coordinates"": [[[9.35820133194473, 47.422598...",St. Gallen,9000,St. Gallen,58.0,45.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW
1,"47.4375957306,9.37816659637","{""coordinates"": [[[9.378254248447286, 47.43760...",St. Gallen,9010,St. Gallen,58.0,43.0,ES III,Wohnen,≤PW,≤PW,≤PW
2,"47.4165304538,9.37865959965","{""coordinates"": [[[9.378730043296102, 47.41652...",St. Gallen,9000,St. Gallen,58.0,43.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW


In [50]:
df.to_csv('daten/Larmbelastung_new.csv', index = False)

## 2. Adressen hinzufügen

In [2]:
# importing modules
from geopy.geocoders import Nominatim

# calling the nominatim tool
geoLoc = Nominatim(user_agent="GetLoc")

from geopy.extra.rate_limiter import RateLimiter


In [14]:
location = geoLoc.reverse(df['Geo Point'].iloc[0])

In [21]:
location.raw

{'place_id': 175125370,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 308304463,
 'lat': '47.433239099999994',
 'lon': '9.40093159928347',
 'display_name': '7, Rehetobelstrasse, St. Fiden, St. Gallen, Wahlkreis St. Gallen, Sankt Gallen, 9000, Schweiz/Suisse/Svizzera/Svizra',
 'address': {'house_number': '7',
  'road': 'Rehetobelstrasse',
  'suburb': 'St. Fiden',
  'town': 'St. Gallen',
  'county': 'Wahlkreis St. Gallen',
  'state': 'Sankt Gallen',
  'postcode': '9000',
  'country': 'Schweiz/Suisse/Svizzera/Svizra',
  'country_code': 'ch'},
 'boundingbox': ['47.4331903', '47.4332878', '9.4007781', '9.4010851']}

In [22]:
location.raw['address']['postcode']

'9000'

In [23]:
#geoLoc = Nominatim(user_agent="application")

reverse = RateLimiter(geoLoc.reverse, min_delay_seconds=5)

addresses_sg = []
mini_dictSG = {'Haus': [], 'Strasse': [], 'PLZ': []}


for i in range(len(df)):
    
    #speichert aufgrund der Koordinaten das Adress-File
    location = geoLoc.reverse(df['Geo Point'].iloc[i])
    
    #holt aus dem Adressen-File die raw-Daten
    loc = location.raw
        
    
#die gewünschten Infos aus den raw-Daten in ein dict

             
    #in den raw-daten kann das Gebäude oder die Hausnummer stehen, so diesen Fall auffangen
    if 'building' in loc['address']:
        mini_dictSG['Haus'].append(loc['address']['building'])
        
    elif 'house_number' in loc['address']:
        mini_dictSG['Haus'].append(loc['address']['house_number'])
    
    #Daten sind teilweise fehlerhaft bzw. lückenhaft
    
    
    try:
        mini_dictSG['Strasse'].append(loc['address']['road'])
    except KeyError:
        mini_dictSG['Strasse'].append('NaN')
        print('error wegen road an der Stelle: ' + str(i))
    
                
    try:
        mini_dictSG['PLZ'].append(loc['address']['postcode'])
    except KeyError:
        mini_dictSG['PLZ'].append('NaN')
        print('error wegen postcode an der Stelle: ' + str(i))
    
    
    finally:
        addresses_sg.append(mini_dictSG)

    
#Infos werden nicht in den mini_dict geschrieben oder sie werden überschrieben. Testen. Sie Code Google

error wegen road an der Stelle: 789
error wegen road an der Stelle: 2022
error wegen road an der Stelle: 3060
error wegen road an der Stelle: 3359
error wegen road an der Stelle: 4428
error wegen road an der Stelle: 5501


In [55]:
df_d = pd.DataFrame.from_dict(mini_dictSG, orient='index')
df_d = df_d.transpose()

In [56]:
df_d

,Object_ID,Haus,Strasse,Dorf,PLZ
0,None,153,Dufourstrasse,None,9000
1,None,68,Guisanstrasse,None,9010
2,None,2,Demutstrasse,None,9000
3,None,60,Kräzernstrasse,None,9015
4,None,279,Zürcher Strasse,None,9014
...,...,...,...,...,...
5594,None,None,Stationsstrasse,None,9014
5595,None,None,Lukasstrasse,None,9016
5596,None,None,Rosenbergweg,None,9000
5597,None,None,Lessingstrasse,None,9008


In [57]:
df_d = df_d.drop(columns=['Object_ID', 'Dorf'])

In [85]:
df_d.tail(208)
# -> Bei den letzten 207 fehlt die Hausnummer (Google-Api)

,Haus,Strasse,PLZ
5391,10,Moosbruggstrasse,9000
5392,None,Martinsbruggstrasse,9016
5393,None,Heiligkreuzstrasse,9008
5394,None,Kräzernstrasse,9015
5395,None,Fuchsenstrasse,9016
...,...,...,...
5594,None,Stationsstrasse,9014
5595,None,Lukasstrasse,9016
5596,None,Rosenbergweg,9000
5597,None,Lessingstrasse,9008


In [62]:
df_d.to_csv('daten/Ergebnisse/larmbelastung-adressen.csv', index = False)

In [59]:
df_new = df.merge(df_d, left_index=True, right_index=True)

In [63]:
df_new.head(30)

,Geo Point,Geo Shape,Gemeinde,PLZ_x,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Haus,Strasse,PLZ_y
0,"47.4225523835,9.35814260469","{""coordinates"": [[[9.35820133194473, 47.422598...",St. Gallen,9000,St. Gallen,58.0,45.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW,153,Dufourstrasse,9000
1,"47.4375957306,9.37816659637","{""coordinates"": [[[9.378254248447286, 47.43760...",St. Gallen,9010,St. Gallen,58.0,43.0,ES III,Wohnen,≤PW,≤PW,≤PW,68,Guisanstrasse,9010
2,"47.4165304538,9.37865959965","{""coordinates"": [[[9.378730043296102, 47.41652...",St. Gallen,9000,St. Gallen,58.0,43.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW,2,Demutstrasse,9000
3,"47.4063236308,9.31706334263","{""coordinates"": [[[9.317027348264398, 47.40638...",St. Gallen,9015,St. Gallen,62.0,48.0,ES III,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW,60,Kräzernstrasse,9015
4,"47.4079375929,9.32924010701","{""coordinates"": [[[9.329150391893581, 47.40797...",St. Gallen,9014,St. Gallen,68.0,61.0,ES III,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",279,Zürcher Strasse,9014
5,"47.4133941377,9.31557847138","{""coordinates"": [[[9.315517223040883, 47.41342...",St. Gallen,9015,St. Gallen,57.0,50.0,ES III,Wohnen,≤PW,≤PW,≤PW,49,Altenwegenstrasse,9015
6,"47.4410248039,9.39314360352","{""coordinates"": [[[9.393265993203421, 47.44103...",St. Gallen,9008,St. Gallen,67.0,60.0,ES III,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",102,Langgasse,9008
7,"47.4244903702,9.38277123037","{""coordinates"": [[[9.382828911688462, 47.42447...",St. Gallen,9000,St. Gallen,64.0,53.0,ES III,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",5,Speicherstrasse,9000
8,"47.4207125948,9.37214922911","{""coordinates"": [[[9.372079541062577, 47.42046...",St. Gallen,9000,St. Gallen,62.0,46.0,ES III,Arbeiten,≤PW,≤PW,≤PW,6,Schochengasse,9000
9,"47.4104515995,9.3620418479","{""coordinates"": [[[9.362127726124314, 47.41049...",St. Gallen,9012,St. Gallen,58.0,46.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",2,Hofstettenstrasse,9012


In [61]:
df_new.to_csv('daten/Ergebnisse/larmbelastung-mit-adresse.csv', index = False)

In [2]:
df = pd.read_csv('daten/Ergebnisse/larmbelastung-mit-adresse.csv')

Einige Stichproben ergeben, dass Nominatim sehr oft nicht die richtige Adresse zugeordnet hat. Daher rufe ich die Koordinaten alle nochmals ab, aber dieses Mal mit der Google API.

### 2.1 Abrufen der Adressen mit Google API

In [54]:
df = pd.read_csv('daten/Larmbelastung_new.csv')

In [55]:
import geopy
from geopy.geocoders import GoogleV3 #der Google-API (Addressabfrage)

from geopy.extra.rate_limiter import RateLimiter #Verzögern der Abfrage

import regex as re

In [56]:
# Google-API geocoding als gelocater setzen (API-Key mitgeben)
geolocator = GoogleV3(api_key = 'XXX')

In [57]:
#RateLlimiter auf zwei Sekunden setzen
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)


addresses = []

for i in range(len(df)):
    #speichert aufgrund der Koordinaten das Adress-File
    location = geolocator.reverse(df['Geo Point'].iloc[i])
    
    #holt aus dem Adressen-File die raw-Daten
    value = location.address
    
    #packt die raw-Daten in eine Liste
    addresses.append(value)

In [58]:
addresses

['Dufourstrasse 153, 9000 St. Gallen, Switzerland',
 'Guisanstrasse 68A, 9010 St. Gallen, Switzerland',
 'Georgshof 2, 9000 St. Gallen, Switzerland',
 'Kräzernstrasse 60, 9015 St. Gallen, Switzerland',
 'Zürcherstrasse 279, 9014 St. Gallen, Switzerland',
 'Altenwegenstrasse 49, 9015 St. Gallen, Switzerland',
 'Langgasse 102, 9008 St. Gallen, Switzerland',
 'Speicherstrasse 5, 9000 St. Gallen, Switzerland',
 'Schochengasse 6, 9000 St. Gallen, Switzerland',
 'Hofstettenstrasse 2, 9012 St. Gallen, Switzerland',
 'Guisanstrasse 66, 9010 St. Gallen, Switzerland',
 'Rosenbergstrasse 42A, 9000 St. Gallen, Switzerland',
 'Gemeinschaftspraxis 2. OG, Oberer Graben 44, 9000 St. Gallen, Switzerland',
 '447 120, 9000 St. Gallen, Switzerland',
 'Lukasstrasse 56, 9016 St. Gallen, Switzerland',
 'Schorenstrasse 51, 9000 St. Gallen, Switzerland',
 'Zwinglistrasse 33, 9000 St. Gallen, Switzerland',
 'Gründenstrasse 34, 9015 St. Gallen, Switzerland',
 'Demutstrasse 5, 9000 St. Gallen, Switzerland',
 'Brü

In [59]:
#mit Regex den Leerschlag zwischen St. und Strassennamen oder Gemeindenamen entfernen
address_new = [re.sub(r'St. ', 'St.', i) for i in addresses]

In [60]:
address_new

['Dufourstrasse 153, 9000 St.Gallen, Switzerland',
 'Guisanstrasse 68A, 9010 St.Gallen, Switzerland',
 'Georgshof 2, 9000 St.Gallen, Switzerland',
 'Kräzernstrasse 60, 9015 St.Gallen, Switzerland',
 'Zürcherstrasse 279, 9014 St.Gallen, Switzerland',
 'Altenwegenstrasse 49, 9015 St.Gallen, Switzerland',
 'Langgasse 102, 9008 St.Gallen, Switzerland',
 'Speicherstrasse 5, 9000 St.Gallen, Switzerland',
 'Schochengasse 6, 9000 St.Gallen, Switzerland',
 'Hofstettenstrasse 2, 9012 St.Gallen, Switzerland',
 'Guisanstrasse 66, 9010 St.Gallen, Switzerland',
 'Rosenbergstrasse 42A, 9000 St.Gallen, Switzerland',
 'Gemeinschaftspraxis 2. OG, Oberer Graben 44, 9000 St.Gallen, Switzerland',
 '447 120, 9000 St.Gallen, Switzerland',
 'Lukasstrasse 56, 9016 St.Gallen, Switzerland',
 'Schorenstrasse 51, 9000 St.Gallen, Switzerland',
 'Zwinglistrasse 33, 9000 St.Gallen, Switzerland',
 'Gründenstrasse 34, 9015 St.Gallen, Switzerland',
 'Demutstrasse 5, 9000 St.Gallen, Switzerland',
 'Brühlgasse 35, 9000 St

In [61]:
address_list = [addresses.split(',') for addresses in address_new]

In [69]:
address_list[4][0]

'Zürcherstrasse 279'

In [70]:
#mit for-loop Adresse und PLZ aus der Liste auslesen und in seperate Listen packen

plz = []
addresse = []

for i in range(len(address_list)):
 
    elem_ad = address_list[i][0]
    addresse.append(elem_ad)
    
    elem_plz = address_list[i][1]
    plz.append(elem_plz)

In [75]:
plz_n = [re.sub(r'St.Gallen', '', i) for i in plz]

In [142]:
#Adresse und PLZ dem df hinzufügen
df['Adresse'] = addresse
df['PLZ_y'] = plz_n

In [143]:
df.head(3)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
0,"47.4225523835,9.35814260469","{""coordinates"": [[[9.35820133194473, 47.422598...",St. Gallen,9000,St. Gallen,58.0,45.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW,Dufourstrasse 153,9000
1,"47.4375957306,9.37816659637","{""coordinates"": [[[9.378254248447286, 47.43760...",St. Gallen,9010,St. Gallen,58.0,43.0,ES III,Wohnen,≤PW,≤PW,≤PW,Guisanstrasse 68A,9010
2,"47.4165304538,9.37865959965","{""coordinates"": [[[9.378730043296102, 47.41652...",St. Gallen,9000,St. Gallen,58.0,43.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW,Georgshof 2,9000


In [144]:
df.shape

(5599, 14)

In [145]:
df.to_csv('daten/Ergebnisse/larmbelastung_all-addresses.csv', index = False)

## 3. Bereinigung

In [141]:
df.sort_values('Adresse', ascending = True)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
769,"47.4264004729,9.38599601018","{""coordinates"": [[[9.38618095782834, 47.426431...",St. Gallen,9000,St. Gallen,60.0,47.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",078 906 83 21,Linsebühlstrasse 105
2624,"47.4264997693,9.38588147294","{""coordinates"": [[[9.386049605920572, 47.42654...",St. Gallen,9000,St. Gallen,60.0,47.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",078 906 83 21,Linsebühlstrasse 105
5462,"47.4149503819,9.36411925541","{""coordinates"": [[[9.36416542643778, 47.414950...",St. Gallen,9000,St. Gallen,50.0,42.0,ES III,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,447 106,9000
474,"47.4146014231,9.36422314363","{""coordinates"": [[[9.3642197982717, 47.4145334...",St. Gallen,9000,St. Gallen,68.0,60.0,ES III,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",447 111,9000
1180,"47.4145920512,9.36362544306","{""coordinates"": [[[9.363661673428531, 47.41451...",St. Gallen,9000,St. Gallen,66.0,58.0,ES III,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",447 114,9000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,"47.4069352736,9.3201650279","{""coordinates"": [[[9.320309485612883, 47.40693...",St. Gallen,9015,St. Gallen,56.0,44.0,ES III,Wohnen,≤PW,≤PW,≤PW,u. Kräzern 2B,9015
964,"47.4075751916,9.32118031634","{""coordinates"": [[[9.321267057371362, 47.40758...",St. Gallen,9015,St. Gallen,43.0,33.0,ES III,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,u. Kräzern 5,9015
3529,"47.4074973495,9.32125683673","{""coordinates"": [[[9.3213827228332, 47.4074734...",St. Gallen,9015,St. Gallen,46.0,34.0,ES III,Arbeiten,≤PW,≤PW,≤PW,u. Kräzern 5,9015
5456,"47.4354207008,9.39606926589","{""coordinates"": [[[9.396131019726138, 47.43544...",St. Gallen,9000,St. Gallen,62.0,52.0,NaN,NaN,Keine ES definiert,Keine ES definiert,Keine ES definiert,u. Lindentalstrasse 104,9000


In [98]:
df.tail(2)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
5597,"47.4367860458,9.38426589577","{""coordinates"": [[[9.384407338843397, 47.43678...",St. Gallen,9008,St. Gallen,59.0,43.0,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",≤PW,Lessingstrasse 24,9008
5598,"47.4230565661,9.35151140158","{""coordinates"": [[[9.351680162920644, 47.42293...",St. Gallen,9000,St. Gallen,57.0,39.0,ES III,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,St.Josefen-Strasse 4,9000


Dies bereinige ich am einfachsten bzw. schnellsten im Excel.

In [4]:
df = pd.read_excel('daten/larmbelastung_all-addressesBEARB.xlsx')

In [182]:
df.to_csv('daten/Ergebnisse/larmbelastung-FINAL.csv', index = False)

Das df enthält Doppelte. Diese sind aber berechtigt:
* Es gibt Häuser, für die mehrere Punkte an der Fassade - an verschiedenen Gebäudeecken - berechnet wurden, daher gibt es für diese Adressen mehrere Angaben. Bsp.:
 * Bionstrasse 1
 * Zwinglistrasse 34
 
* Es gibt Adressen, denen mehrere Gebäude zugeorndet sind. Bestes Beispiel ist hier das Kantonsspital mit der Adresse Rorschacher Strasse 95, diese kommt daher 12-mal vor.

-> Über die Koordinaten in der Spalte Geo Point ist ersichtlich, auf welche Gebäudeecke bzw. welches Gebäude einer Adresse sich die Daten beziehen.

In [5]:
df[df['Adresse'].duplicated() == True].sort_values('Adresse')

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,Empfindlichkeitsstufe (ES),Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
43,"47.4347144167,9.38769324738","{""coordinates"": [[[9.387693105681945, 47.43477...",St. Gallen,9008,St. Gallen,58,52,ES II,Wohnen,">IGW, <AW",">PW, ≤IGW",">IGW, <AW",Aeplistrasse 10,9008
67,"47.4347758077,9.38867253627","{""coordinates"": [[[9.38868653374333, 47.434826...",St. Gallen,9008,St. Gallen,64,56,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Aepliweg 1,9008
86,"47.4128765641,9.34833434843","{""coordinates"": [[[9.34833780090121, 47.412820...",St. Gallen,9000,St. Gallen,57,43,ES III,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Ahornstrasse 41,9000
96,"47.4104667355,9.31274385905","{""coordinates"": [[[9.312814344745824, 47.41052...",St. Gallen,9015,St. Gallen,60,42,ES III,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Altenwegenstrasse 25,9015
97,"47.4102886411,9.31281339137","{""coordinates"": [[[9.31289146603397, 47.410341...",St. Gallen,9015,St. Gallen,57,47,ES III,Wohnen,≤PW,≤PW,≤PW,Altenwegenstrasse 25,9015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5491,"47.4070942421,9.29725633533","{""coordinates"": [[[9.297775932451684, 47.40712...",St. Gallen,9015,St. Gallen,68,48,ES IV,Arbeiten,">PW, ≤IGW",">PW, ≤IGW",≤PW,Zürcher Strasse 499,9015
5497,"47.4068538174,9.2943332259","{""coordinates"": [[[9.294754711596362, 47.40715...",St. Gallen,9015,St. Gallen,67,53,ES IV,Arbeiten,">PW, ≤IGW",">PW, ≤IGW",≤PW,Zürcher Strasse 511,9015
5511,"47.4174565692,9.35092939853","{""coordinates"": [[[9.350933824714769, 47.41741...",St. Gallen,9000,St. Gallen,64,36,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Zürcher Strasse 65,9000
5525,"47.4167166468,9.34822390052","{""coordinates"": [[[9.348282987455732, 47.41674...",St. Gallen,9000,St. Gallen,69,61,ES III,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Zürcher Strasse 90,9000


## 4. Auswertung

In [6]:
df.rename({
    'Empfindlichkeitsstufe (ES)' : 'ES'
    }, axis = 1, inplace = True)

### 4.1 An welchen Häusern ist es in der Stadt St.Gallen Tag/Nacht am lautesten/am stillsten

Wo ist es am allerlautesten?

In [7]:
#Wo ist es am lautesten, am Tag und in der Nacht?
df[['Immissionswert Tag', 'Immissionswert Nacht']].max()

Immissionswert Tag      78
Immissionswert Nacht    72
dtype: int64

In [8]:
#Am lautesten ist es am Tag und in der Nacht mit 78 resp. 72 Dezibeln (wenig überraschend) an der Autobahn:
df[df['Immissionswert Tag'] >= 78]

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
3669,"47.4483838726,9.42056294062","{""coordinates"": [[[9.420442777403608, 47.44847...",St. Gallen,9016,St. Gallen,78,72,ES III,Wohnen,≥AW,≥AW,≥AW,Rorschacher Strasse 308,9016


Nach Nutzung Wohnen, am Tag:

In [9]:
#Und wo ist es nun am lautesten bei einem Haus AM TAG?
df[(df['Immissionswert Tag'] >= 75) & (df['Nutzung'] == 'Wohnen')]

# 78 Dezibel am Tag und 72 Dezibel in der Nacht bei dem alten Bauernhaus direkt an der A1 (siehe codezeile oben).
# 75 Dezibel am Tag und 69 Dezibel in der Nacht im (eichentlich) Gewerbegebiet hinter einer Kirche (Altenwegenstrasse 117 in 9015)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
94,"47.4180701809,9.32904067346","{""coordinates"": [[[9.328953641211287, 47.41802...",St. Gallen,9015,St. Gallen,75,69,ES III,Wohnen,≥AW,≥AW,≥AW,Altenwegenstrasse 117,9015
3669,"47.4483838726,9.42056294062","{""coordinates"": [[[9.420442777403608, 47.44847...",St. Gallen,9016,St. Gallen,78,72,ES III,Wohnen,≥AW,≥AW,≥AW,Rorschacher Strasse 308,9016


Nach Nutzung Wohnen, in der Nacht:

In [10]:
# Gibt es andere Werte zu einem Wohngebiet IN DER NACHT?
df[(df['Immissionswert Nacht'] >= 65) & (df['Nutzung'] == 'Wohnen')].sort_values('Immissionswert Nacht', ascending = False)

#Im Industriegebiet hinter einer Kirche

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
3669,"47.4483838726,9.42056294062","{""coordinates"": [[[9.420442777403608, 47.44847...",St. Gallen,9016,St. Gallen,78,72,ES III,Wohnen,≥AW,≥AW,≥AW,Rorschacher Strasse 308,9016
94,"47.4180701809,9.32904067346","{""coordinates"": [[[9.328953641211287, 47.41802...",St. Gallen,9015,St. Gallen,75,69,ES III,Wohnen,≥AW,≥AW,≥AW,Altenwegenstrasse 117,9015
5483,"47.4082951327,9.30534321069","{""coordinates"": [[[9.306854066609267, 47.40830...",St. Gallen,9015,St. Gallen,74,67,ES III,Wohnen,≥AW,≥AW,≥AW,Zürcher Strasse 462,9015
3356,"47.4113440591,9.31189969367","{""coordinates"": [[[9.312369497313055, 47.41188...",St. Gallen,9015,St. Gallen,72,66,ES III,Wohnen,≥AW,≥AW,≥AW,Piccardstrasse 19,9015
4012,"47.4396447798,9.41281452324","{""coordinates"": [[[9.412759445325849, 47.43972...",St. Gallen,9016,St. Gallen,72,65,ES III,Wohnen,≥AW,≥AW,≥AW,Schönbüelpark 12,9016


In [11]:
df_larmTAG = df.sort_values('Immissionswert Tag', ascending = False).head(10)

In [12]:
df_larmTAG.to_csv('daten/Ergebnisse/larmbelastung-top_ten-TAG.csv', index = False)

In [13]:
df_larmNACHT = df.sort_values('Immissionswert Nacht', ascending = False).head(10)

In [14]:
df_larmNACHT.to_csv('daten/Ergebnisse/larmbelastung-top_ten-NACHT.csv', index = False)

Nach Empfindlichkeitsstufe II = Wohnen, am Tag:

In [15]:
#Wie sieht die Situation nach ES aus? Wo ist es in der Wohnzone (ESII) am lautesten AM TAG?
df[(df['Immissionswert Tag'] >= 73) & (df['ES'] == 'ES II')]

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
4357,"47.4326517362,9.38486402844","{""coordinates"": [[[9.384776712073144, 47.43261...",St. Gallen,9000,St. Gallen,73,53,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Splügenstrasse 12,9008
5485,"47.4080342444,9.30200492013","{""coordinates"": [[[9.301948671057657, 47.40806...",St. Gallen,9015,St. Gallen,73,59,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Zürcher Strasse 475,9015


Nach Empfindlichkeitsstufe II = Wohnen, in der Nacht:

In [16]:
#Wie sieht die Situation nach ES aus? Wo ist es in der Wohnzone (ESII) am lautesten IN DER NACHT?
df[(df['Immissionswert Nacht'] >= 60) & (df['ES'] == 'ES II')].sort_values('Immissionswert Nacht', ascending = False)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
2105,"47.4276828605,9.37548838778","{""coordinates"": [[[9.375767300424071, 47.42763...",St. Gallen,9000,St. Gallen,69,62,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Kirchgasse 15,9000
5476,"47.4074524742,9.30891289222","{""coordinates"": [[[9.308960575048712, 47.40748...",St. Gallen,9015,St. Gallen,69,62,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Zürcher Strasse 436,9015
5308,"47.4157789855,9.34468538565","{""coordinates"": [[[9.344956621796944, 47.41592...",St. Gallen,9000,St. Gallen,68,61,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Zürcher Strasse 117,9000
4360,"47.4343000155,9.38805838318","{""coordinates"": [[[9.38822630616293, 47.434321...",St. Gallen,9008,St. Gallen,67,60,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Splügenstrasse 21,9008


In [17]:
df_wohnen = df[df['ES'] == 'ES II'].copy()

In [18]:
df_larmTAG = df_wohnen.sort_values('Immissionswert Tag', ascending = False).head(10)

In [19]:
df_larmTAG.to_csv('daten/Ergebnisse/larmbelastung_ESII_top-tenTAG.csv', index = False)

In [20]:
df_larmNACHT = df_wohnen.sort_values('Immissionswert Nacht', ascending = False).head(10)

In [21]:
df_larmNACHT.to_csv('daten/Ergebnisse/larmbelastung_ESII_top-tenNACHT.csv', index = False)

Wo ist es am allerleisesten?

In [22]:
#Wo ist es am leisesten, am Tag und in der Nacht?
df[['Immissionswert Tag', 'Immissionswert Nacht']].min()

Immissionswert Tag      34
Immissionswert Nacht    15
dtype: int64

In [23]:
#Das ist lustig: Am leisesten ist es am Tag mit 34 Dezibeln (wohl über dem) Rosenbergtunnel der Autobahn:
df[df['Immissionswert Tag'] == 34]

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
338,"47.4308642426,9.37295221268","{""coordinates"": [[[9.372991739616625, 47.43085...",St. Gallen,9000,St. Gallen,34,24,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Bodanstrasse 8,9000


In [24]:
# Und in der Nacht ist es im Quartier St.Georgen, etwas abseits der dortigen Hauptachse, am ruhigsten, 15 Dezibel:
df[df['Immissionswert Nacht'] == 15]

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y
4434,"47.4142444406,9.39206765614","{""coordinates"": [[[9.39213152881801, 47.414173...",St. Gallen,9011,St. Gallen,44,15,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,St.Georgen-Strasse 166,9011


### 4.2 Einteilung nach Grenzwerten (Tag/Nacht) für ESII

Mal nur für die Wohngebiete (Empfindlichkeitsstuffe II), soweit Nutzung vorhanden.

In [25]:
#Bei wie vielen fehlt die Nutzung?

df['Nutzung'].isna().sum()

860

In [26]:
#Bei wie vielen fehlt die ES?

df['ES'].isna().sum()

10

In [27]:
df_wohnen = df[df['ES'] == 'ES II'].copy()
df_wohnen.shape

(2976, 14)

In [28]:
df['ES'].value_counts()

ES II     2976
ES III    2545
ES IV       60
Name: ES, dtype: int64

In [29]:
df['Nutzung'].value_counts()

Wohnen                          4238
Arbeiten                         407
Schulen, Anstalten und Heime      86
Name: Nutzung, dtype: int64

In [30]:
#csv mit Grenzwerten einlesen
df_grenzwerte = pd.read_csv('daten/grenzwerte.csv')
df_grenzwerte

,Empfindlichkeitsstufe,tageszeit,planungswert,Grenzwert,alarmwert
0,Erholung,Tag,50,55,65
1,Erholung,Nacht,40,45,60
2,Wohnen,Tag,55,60,70
3,Wohnen,Nacht,45,50,65
4,Wohnen/Gewerbe,Tag,60,65,70
5,Wohnen/Gewerbe,Nacht,50,55,65
6,Industrie,Tag,65,70,75
7,Industrie,Nacht,55,60,70


In [31]:
import numpy as np

In [32]:
#Den Planungswert ignoriere ich, da der nur für Bauprojekte bzw. Einzonungen relevant ist.

bin_values_tag = [
    0,
    59,
    69,
    np.inf]

bin_values_nacht = [
    0,
    49,
    64,
    np.inf]

In [33]:
bin_label_tag = [
    "1. unter 60 Dezibel",
    "2. 60 bis 69 Dezibel",
    "3. über 70 Dezibel"]
bin_label_nacht = [
    "1. unter 50 Dezibel",
    "2. 50 bis 64 Dezibel",
    "3. über 65 Dezibel"]

colors_tag = {
    "1. unter 60 Dezibel" : "yellow",
    "2. 60 bis 69 Dezibel" : "orange",
    "3. über 70 Dezibel": "red"}

colors_nacht = {
    "1. unter 50 Dezibel" : "yellow",
    "2. 50 bis 64 Dezibel" : "orange",
    "3. über 65 Dezibel" : "red"}

In [34]:
df_wohnen['Kategorie Immissionswert Tag'] = pd.cut(df_wohnen['Immissionswert Tag'], bin_values_tag, labels=bin_label_tag, include_lowest=True).astype(str)

In [35]:
df_wohnen['Kategorie Immissionswert Nacht'] = pd.cut(df_wohnen['Immissionswert Nacht'], bin_values_nacht, labels=bin_label_nacht, include_lowest=True).astype(str)

In [36]:
df_wohnen.groupby('Kategorie Immissionswert Tag')['Immissionswert Tag'].max()

Kategorie Immissionswert Tag
1. unter 60 Dezibel     59
2. 60 bis 69 Dezibel    69
3. über 70 Dezibel      73
Name: Immissionswert Tag, dtype: int64

In [37]:
df_wohnen.groupby('Kategorie Immissionswert Nacht')['Immissionswert Nacht'].max()

Kategorie Immissionswert Nacht
1. unter 50 Dezibel     49
2. 50 bis 64 Dezibel    62
Name: Immissionswert Nacht, dtype: int64

In [38]:
df_wohnen.head(5)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y,Kategorie Immissionswert Tag,Kategorie Immissionswert Nacht
1,"47.4264004729,9.38599601018","{""coordinates"": [[[9.38618095782834, 47.426431...",St. Gallen,9000,St. Gallen,60,47,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",Linsebühlstrasse 105,9000,2. 60 bis 69 Dezibel,1. unter 50 Dezibel
2,"47.4264997693,9.38588147294","{""coordinates"": [[[9.386049605920572, 47.42654...",St. Gallen,9000,St. Gallen,60,47,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",Singenbergstrasse 26,9000,2. 60 bis 69 Dezibel,1. unter 50 Dezibel
26,"47.4327422272,9.41220262154","{""coordinates"": [[[9.412280839456592, 47.43267...",St. Gallen,9016,St. Gallen,55,41,ES II,Wohnen,≤PW,≤PW,≤PW,Achslenstrasse 14,9016,1. unter 60 Dezibel,1. unter 50 Dezibel
27,"47.4326199723,9.41152910048","{""coordinates"": [[[9.411636125899776, 47.43263...",St. Gallen,9016,St. Gallen,53,40,ES II,Wohnen,≤PW,≤PW,≤PW,Achslenstrasse 18,9016,1. unter 60 Dezibel,1. unter 50 Dezibel
28,"47.4327704576,9.41140609279","{""coordinates"": [[[9.411506654684569, 47.43276...",St. Gallen,9016,St. Gallen,54,40,ES II,Wohnen,≤PW,≤PW,≤PW,Achslenstrasse 20,9016,1. unter 60 Dezibel,1. unter 50 Dezibel


In [39]:
df_wohnen.sort_values(['Kategorie Immissionswert Tag', 'Immissionswert Tag'], ascending = [True, True], inplace = True)

In [40]:
df_wohnen.to_csv('daten/Ergebnisse/larmbelastung_df_wohnen_.csv', index = False)

In [41]:
df_wohnen = pd.read_csv('daten/Ergebnisse/larmbelastung_df_wohnen_.csv')

#### Wie oft Grenz- bzw. Alarmwert erreicht in ESII

In [42]:
df_wohnen['Kategorie Immissionswert Tag'].value_counts()

1. unter 60 Dezibel     2415
2. 60 bis 69 Dezibel     557
3. über 70 Dezibel         4
Name: Kategorie Immissionswert Tag, dtype: int64

In [43]:
df_wohnen['Kategorie Immissionswert Nacht'].value_counts()

1. unter 50 Dezibel     2614
2. 50 bis 64 Dezibel     362
Name: Kategorie Immissionswert Nacht, dtype: int64

#### Wo wird der Grenzwert erreicht

Beim df_wohnen: Wenn in der Nacht 50 und am Tag 60 Dezibel überschritten = Grenzwert erreicht. Wenn in der Nacht 65 und am Tag 70 Dezibel überschritten = Alarmwert erreicht.

am Tag:

In [44]:
df_wohnen[df_wohnen['Kategorie Immissionswert Tag'] == '2. 60 bis 69 Dezibel'].sort_values('Immissionswert Nacht', ascending = False)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y,Kategorie Immissionswert Tag,Kategorie Immissionswert Nacht
2971,"47.4074524742,9.30891289222","{""coordinates"": [[[9.308960575048712, 47.40748...",St. Gallen,9015,St. Gallen,69,62,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Zürcher Strasse 436,9015,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2970,"47.4276828605,9.37548838778","{""coordinates"": [[[9.375767300424071, 47.42763...",St. Gallen,9000,St. Gallen,69,62,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Kirchgasse 15,9000,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2969,"47.4157789855,9.34468538565","{""coordinates"": [[[9.344956621796944, 47.41592...",St. Gallen,9000,St. Gallen,68,61,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Zürcher Strasse 117,9000,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2956,"47.4343000155,9.38805838318","{""coordinates"": [[[9.38822630616293, 47.434321...",St. Gallen,9008,St. Gallen,67,60,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Splügenstrasse 21,9008,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2962,"47.4073848765,9.3181062735","{""coordinates"": [[[9.318173134134291, 47.40744...",St. Gallen,9015,St. Gallen,67,59,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Zürcher Strasse 391,9015,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2573,"47.4251039043,9.38086289133","{""coordinates"": [[[9.381019063275343, 47.42556...",St. Gallen,9000,St. Gallen,61,30,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Burggraben 21,9000,2. 60 bis 69 Dezibel,1. unter 50 Dezibel
2423,"47.4462727764,9.39575737714","{""coordinates"": [[[9.395839377204913, 47.44632...",St. Gallen,9008,St. Gallen,60,29,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Bruggwaldstrasse 41,9008,2. 60 bis 69 Dezibel,1. unter 50 Dezibel
2524,"47.4162935456,9.38248957626","{""coordinates"": [[[9.382449321087494, 47.41641...",St. Gallen,9011,St. Gallen,60,28,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,St.Georgen-Strasse 89,9000,2. 60 bis 69 Dezibel,1. unter 50 Dezibel
2805,"47.425142599,9.38692804334","{""coordinates"": [[[9.38697629664466, 47.425085...",St. Gallen,9000,St. Gallen,63,27,ES II,"Schulen, Anstalten und Heime",">IGW, <AW",">IGW, <AW",≤PW,Molkenstrasse 1,9000,2. 60 bis 69 Dezibel,1. unter 50 Dezibel


in der Nacht:

In [45]:
df_wohnen[df_wohnen['Kategorie Immissionswert Nacht'] == '2. 50 bis 64 Dezibel'].sort_values('Immissionswert Nacht', ascending = False)

,Geo Point,Geo Shape,Gemeinde,PLZ,Wahlkreis,Immissionswert Tag,Immissionswert Nacht,ES,Nutzung,Lärmbeurteilung,Lärmbeurteilung Tag,Lärmbeurteilung Nacht,Adresse,PLZ_y,Kategorie Immissionswert Tag,Kategorie Immissionswert Nacht
2971,"47.4074524742,9.30891289222","{""coordinates"": [[[9.308960575048712, 47.40748...",St. Gallen,9015,St. Gallen,69,62,ES II,NaN,Keine Nutzung definiert,Keine Nutzung definiert,Keine Nutzung definiert,Zürcher Strasse 436,9015,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2970,"47.4276828605,9.37548838778","{""coordinates"": [[[9.375767300424071, 47.42763...",St. Gallen,9000,St. Gallen,69,62,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Kirchgasse 15,9000,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2969,"47.4157789855,9.34468538565","{""coordinates"": [[[9.344956621796944, 47.41592...",St. Gallen,9000,St. Gallen,68,61,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Zürcher Strasse 117,9000,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2956,"47.4343000155,9.38805838318","{""coordinates"": [[[9.38822630616293, 47.434321...",St. Gallen,9008,St. Gallen,67,60,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Splügenstrasse 21,9008,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2962,"47.4073848765,9.3181062735","{""coordinates"": [[[9.318173134134291, 47.40744...",St. Gallen,9015,St. Gallen,67,59,ES II,Wohnen,">IGW, <AW",">IGW, <AW",">IGW, <AW",Zürcher Strasse 391,9015,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,"47.436318036,9.41931455119","{""coordinates"": [[[9.419395013850698, 47.43640...",St. Gallen,9016,St. Gallen,59,50,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",Vogelherdstrasse 7C,9016,1. unter 60 Dezibel,2. 50 bis 64 Dezibel
2410,"47.3997244385,9.29888701809","{""coordinates"": [[[9.298988650818726, 47.39969...",St. Gallen,9015,St. Gallen,59,50,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",Zeppelinstrasse 8,9015,1. unter 60 Dezibel,2. 50 bis 64 Dezibel
2424,"47.4334189113,9.39871743384","{""coordinates"": [[[9.398719488878264, 47.43349...",St. Gallen,9000,St. Gallen,60,50,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",Buchentalstrasse 4,9000,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel
2438,"47.413828971,9.34013305088","{""coordinates"": [[[9.340185685012045, 47.41386...",St. Gallen,9014,St. Gallen,60,50,ES II,Wohnen,">PW, ≤IGW",">PW, ≤IGW",">PW, ≤IGW",Eisbahnstrasse 14,9014,2. 60 bis 69 Dezibel,2. 50 bis 64 Dezibel


### 4.3 Einteilung nach Grenzwerten (Tag/Nacht) für ESII

In [46]:
df_misch = df[df['ES'] == 'ES III'].copy()
df_misch.shape

(2545, 14)

In [47]:
#Den Planungswert ignoriere ich, da der nur für Bauprojekte bzw. Einzonungen relevant ist.

bin_values_tag = [
    0,
    64,
    69,
    np.inf]

bin_values_nacht = [
    0,
    54,
    64,
    np.inf]

In [48]:
bin_label_tag = [
    "1. unter 65 Dezibel",
    "2. 65 bis 69 Dezibel",
    "3. über 70 Dezibel"]
bin_label_nacht = [
    "1. unter 55 Dezibel",
    "2. 55 bis 64 Dezibel",
    "3. über 65 Dezibel"]

In [49]:
df_misch['Kategorie Immissionswert Tag'] = pd.cut(df_misch['Immissionswert Tag'], bin_values_tag, labels=bin_label_tag, include_lowest=True).astype(str)

In [50]:
df_misch['Kategorie Immissionswert Nacht'] = pd.cut(df_misch['Immissionswert Nacht'], bin_values_nacht, labels=bin_label_nacht, include_lowest=True).astype(str)

In [51]:
df_misch.groupby('Kategorie Immissionswert Tag')['Immissionswert Tag'].max()

Kategorie Immissionswert Tag
1. unter 65 Dezibel     64
2. 65 bis 69 Dezibel    69
3. über 70 Dezibel      78
Name: Immissionswert Tag, dtype: int64

In [52]:
df_misch.groupby('Kategorie Immissionswert Nacht')['Immissionswert Nacht'].max()

Kategorie Immissionswert Nacht
1. unter 55 Dezibel     54
2. 55 bis 64 Dezibel    64
3. über 65 Dezibel      72
Name: Immissionswert Nacht, dtype: int64

In [53]:
df_misch.to_csv('daten/Ergebnisse/larmbelastung_df_misch_.csv', index = False)

#### Wie oft Grenz- bzw. Alarmwert erreicht in ESIII

In [54]:
df_misch['Kategorie Immissionswert Tag'].value_counts()

1. unter 65 Dezibel     1828
2. 65 bis 69 Dezibel     680
3. über 70 Dezibel        37
Name: Kategorie Immissionswert Tag, dtype: int64

In [55]:
df_misch['Kategorie Immissionswert Nacht'].value_counts()

1. unter 55 Dezibel     1736
2. 55 bis 64 Dezibel     802
3. über 65 Dezibel         7
Name: Kategorie Immissionswert Nacht, dtype: int64

### 4.4 Minimal- und Maximalwerte pro PLZ und Durschnittswerte (In welchen Gebieten ist es am lautesten)

In [56]:
#Wichtig zu wissen: Wie viele Werte liegen pro PLZ vor:
df['PLZ_y'].value_counts()

9000    3019
9016     676
9008     648
9014     443
9015     360
9011     163
9010     121
9012     120
9007      11
9001       8
9037       7
9300       4
9030       4
9042       4
9004       2
9013       1
Name: PLZ_y, dtype: int64

In [57]:
#Neues df mit dem Maximalwert und Durchschnitt pro PLZ

df_plz = df.groupby(['PLZ_y', 'ES'])['Immissionswert Tag'].mean().to_frame()

In [58]:
df_plz

Immissionswert Tag
PLZ_y ES                        
9000  ES II            55.337475
      ES III           60.938005
9001  ES II            60.500000
      ES III           63.500000
9004  ES II            63.000000
9007  ES II            56.181818
9008  ES II            55.685131
      ES III           61.376238
9010  ES II            55.036697
      ES III           57.666667
9011  ES II            56.304965
      ES III           58.590909
9012  ES II            55.657534
      ES III           62.065217
9013  ES III           62.000000
9014  ES II            54.771429
      ES III           60.634328
9015  ES II            55.378151
      ES III           60.777251
      ES IV            61.566667
9016  ES II            54.280172
      ES III           61.406593
      ES IV            64.200000
9030  ES II            63.000000
      ES III           67.500000
9037  ES III           59.500000
9042  ES III           60.000000
9300  ES II            59.500000

In [59]:
df_plz['max_tag'] = df.groupby(['PLZ_y', 'ES'])['Immissionswert Tag'].max()

In [60]:
df_plz['min_tag'] = df.groupby(['PLZ_y', 'ES'])['Immissionswert Tag'].min()

In [61]:
df_plz.rename({'Immissionswert Tag' : 'Schnitt Immissionswert Tag'},
              axis = 1, inplace = True)

In [62]:
df_plz.reset_index(inplace = True)

In [63]:
df_plz.to_csv('daten/Ergebnisse/df_plz_larmbelastung_TAG.csv', index = False)

In [64]:
df_plz_n = df.groupby(['PLZ_y', 'ES'])['Immissionswert Nacht'].max().to_frame()

In [65]:
df_plz_n['min_nacht'] = df.groupby(['PLZ_y', 'ES'])['Immissionswert Nacht'].min()

In [66]:
df_plz_n['Schnitt Immissionswert Nacht'] = df.groupby(['PLZ_y', 'ES'])['Immissionswert Nacht'].mean()

In [67]:
df_plz_n

Immissionswert Nacht  min_nacht  Schnitt Immissionswert Nacht
PLZ_y ES                                                                   
9000  ES II                     62         17                     43.198169
      ES III                    63         21                     50.197439
9001  ES II                     53         33                     44.750000
      ES III                    52         37                     45.500000
9004  ES II                     51         41                     46.000000
9007  ES II                     55         37                     45.545455
9008  ES II                     60         26                     43.970845
      ES III                    62         31                     51.811881
9010  ES II                     46         29                     40.055046
      ES III                    43         39                     41.833333
9011  ES II                     51         15                     40.964539
      ES III                    49         19                     40.954545
9012  ES II                     58         18                     44.136986
      ES III                    61         31                     52.347826
9013  ES III                    28         28                     28.000000
9014  ES II                     59         25                     43.034286
      ES III                    62         23                     49.432836
9015  ES II                     62         26                     44.252101
      ES III                    69         25                     46.777251
      ES IV                     64         34                     46.500000
9016  ES II                     59         27                     43.956897
      ES III                    72         22                     49.758242
      ES IV                     62         30                     44.400000
9030  ES II                     55         42                     48.500000
      ES III                    61         56                     58.500000
9037  ES III                    52         24                     38.333333
9042  ES III                    49         44                     46.000000
9300  ES II                     53         47                     50.250000

In [68]:
df_plz_n.reset_index(inplace = True)

In [69]:
df_plz_n.rename({'Immissionswert Nacht' : 'min_Nacht'},
              axis = 1, inplace = True)

In [70]:
df_plz_n['Schnitt Immissionswert Nacht'].sort_values(ascending = False)

24    58.500000
13    52.347826
7     51.811881
27    50.250000
1     50.197439
21    49.758242
16    49.432836
23    48.500000
18    46.777251
19    46.500000
26    46.000000
4     46.000000
5     45.545455
3     45.500000
2     44.750000
22    44.400000
17    44.252101
12    44.136986
6     43.970845
20    43.956897
0     43.198169
15    43.034286
9     41.833333
10    40.964539
11    40.954545
8     40.055046
25    38.333333
14    28.000000
Name: Schnitt Immissionswert Nacht, dtype: float64

In [72]:
df_plz_n.to_csv('daten/Ergebnisse/df_plz_larmbelastung_NACHT.csv', index = False)